### Dataset info
https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)

The analysis will contemplate the prediction of a certain cancer, indicating whether it is benign (0) or malignant (1).

### Installing required modules

In [1]:
!pip install pandas
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install h5py

### Importing data

In [2]:
import pandas as pd

In [3]:
features = pd.read_csv('datasets/breast_cancer_wisconsin_diagnostic_data.csv')

In [4]:
classes = pd.read_csv('datasets/breast_cancer_wisconsin_diagnostic_classes.csv')

### Building the Neural Network with Dropout

In [5]:
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

2023-01-21 21:08:42.898890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 21:08:43.059365: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-21 21:08:43.059383: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-21 21:08:43.818921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
input_dim = features.shape[1]
input_units = 16
output_units = 1

model = Sequential()
model.add(Dense(units=input_units, activation='relu', kernel_initializer='random_uniform', input_dim=input_dim))
model.add(Dropout(0.2))
model.add(Dense(units=input_units, activation='relu', kernel_initializer='random_uniform'))
model.add(Dropout(0.2))
model.add(Dense(units=output_units, activation='sigmoid'))

adam_optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

2023-01-21 21:08:44.654231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-21 21:08:44.657850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-21 21:08:44.658062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-21 21:08:44.658137: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-21 21:08:44.658192: W tensorflow/c

### Training

In [7]:
model.fit(features, classes)

18/18 [==============================] - 1s 8ms/step - loss: 1.6183 - binary_accuracy: 0.5132


### Save model

In [8]:
model_structure_json = model.to_json()

In [9]:
with open('files/breast_cancer_diagnostic_structure.json', 'w') as f:
    f.write(model_structure_json)

In [10]:
model.save_weights('files/breast_cancer_diagnostic_weights.h5')

### Restore model

In [11]:
from keras.models import model_from_json

In [12]:
f = open('files/breast_cancer_diagnostic_structure.json', 'r')
model_structure_json = f.read()
f.close()

In [13]:
restored_model = model_from_json(model_structure_json)

In [14]:
restored_model.load_weights('files/breast_cancer_diagnostic_weights.h5')

### Testing

In [15]:
import numpy as np

In [16]:
to_predict = np.array([[
    15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178, 0.20,
    0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015, 0.03,
    0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185, 0.84, 158, 0.363
]])

In [17]:
predicted = restored_model.predict(to_predict)

1/1 [==============================] - 0s 214ms/step


In [18]:
predicted = (predicted > 0.5)

In [19]:
print(f'Predicted: {predicted}')

Predicted: [[ True]]


In [20]:
adam_optimizer = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)
restored_model.compile(optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

In [21]:
result = restored_model.evaluate(features, classes)

18/18 [==============================] - 1s 9ms/step - loss: 0.5354 - binary_accuracy: 0.6591


In [22]:
print(f'Accuracy: {result[1] * 100:.2f}')

Accuracy: 65.91
